# **Description**

---

Compares socio-economic info with suicide rates by year and country.

References
United Nations Development Program. (2018). Human development index (HDI). Retrieved from http://hdr.undp.org/en/indicators/137506

World Bank. (2018). World development indicators: GDP (current US$) by country:1985 to 2016. Retrieved from http://databank.worldbank.org/data/source/world-development-indicators#

[Szamil]. (2017). Suicide in the Twenty-First Century [dataset]. Retrieved from https://www.kaggle.com/szamil/suicide-in-the-twenty-first-century/notebook

World Health Organization. (2018). Suicide prevention. Retrieved from http://www.who.int/mental_health/suicide-prevention/en/

Inspiration
Suicide Prevention.




# **Data Description**
**country** - Name of the Country

**year** - year of the sample

**sex** - Male of Famle

**age group** - range of age

**count of suicides** - number of suicides for this year

**population** - the size of the population

**suicide rate** -suicides per 100k population

**country-year** composite key - combination of country and year

**HDI for year** - Human development index (HDI)

**gdp_for_year** - Gross domestic product (GDP) - Gross domestic product (GDP) is a monetary measure of the market value of all the final goods and services produced in a specific time period, often annually. 
**GDP = C + I + G + (eX - i)**

C = total spending by consumers

I = total investment (spending on goods and services) by businesses

G = total spending by government (federal, state, and local)

(Ex - Im) = net exports (exports - imports)

**gdp_per_capita** - This is the value of all final goods and services produced within a nation in a given year, converted at market exchange rates to current U.S. dollars, divided by the average population for the same year.

**generation (based on age grouping average)** - - the generation based on year 


# <a id='load_libraries'>Load the libraries</a>



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
from matplotlib import rcParams
%matplotlib inline

In [ ]:
Suicide = pd.read_csv('../input/suicide-rates-overview-1985-to-2016/master.csv')

In [ ]:
Suicide.info()

In [ ]:
Suicide.head()

In [ ]:
print("Maximum number of samples per country:        ",Suicide.groupby('country')['country'].count().max(),Suicide.groupby('country')['country'].count().idxmax() )
print("Minimum number of samples per country:        ",Suicide.groupby('country')['country'].count().min(), Suicide.groupby('country')['country'].count().idxmin())
print("Total number of countries in the sample:      ",Suicide['country'].nunique())
print("Total number of unique rows (country + year) in the sample: ",Suicide['country-year'].nunique())

In [ ]:
Suicide[Suicide['country'] == 'Mongolia']

There are 101 countries in this study. Each country has different number of samples. It means that we do not have the suicide data for all countries for all years. For example, Mongolia has only 10 samples from 2016.



In [ ]:
import missingno
print(Suicide.isna().sum())
print("\n\nPrecentage of NULLs of the HDI for year: ", Suicide['HDI for year'].isna().sum()/Suicide.shape[0])
missingno.matrix(Suicide, figsize = (30,10))

70% of the HDI for year feature is NULL value. Probably need to avoid this feature.

In [ ]:
nnull = Suicide[Suicide['HDI for year'].isna() == False]
nnull.shape[0]
rcParams['figure.figsize'] = 10,3
#sns.scatterplot(x="gdp_per_capita ($)", y="HDI for year", data=nnull)
sns.lmplot(x="gdp_per_capita ($)", y="HDI for year",data=nnull,logx=True,ci=None , scatter_kws={"s" : 70})

In [ ]:
Suicide.describe()

As can be seen in the graph above there is an exponent relation between 'HDI for year' and 'GDP per capita'. So we can use GDP per capita feature as a welth indicator instead of HDI.

# <a id='load_libraries'>Number of Suicides per Country</a>

In [ ]:
sortSu = Suicide.groupby('country').mean()
sortSu['Country'] = sortSu.index
rcParams['figure.figsize'] = 10,3
sortSu = sortSu.sort_values('suicides/100k pop') 
dis = sns.barplot(x=sortSu['Country'][-20:], y=sortSu['suicides/100k pop'][-20:], palette="rocket")
plt.title('Average number of Suicide per Country - Top 20', fontsize=20)

rcParams['figure.figsize'] = 12,3

for item in dis.get_xticklabels():
    item.set_rotation(45)
    
dis.plot()

In [ ]:
dis = sns.barplot(x=sortSu['Country'][0:20], y=sortSu['suicides/100k pop'][0:20], palette="rocket")
plt.title('Average number of Suicide per Country - bottom 20', fontsize=20)

rcParams['figure.figsize'] = 10,3

for item in dis.get_xticklabels():
    item.set_rotation(45)
    
dis.plot()

From the above graphs we can obsorve that there is a big gap between the average top 20 and teh avergae botton 20. We need to try and identify if we can find any explanation.

# <a id='load_libraries'>Distribution of normelized number of Suicides</a>

In [ ]:
rcParams['figure.figsize'] = 8,3
x = sortSu[sortSu['suicides/100k pop'] > 16]
sns.distplot(x['suicides/100k pop'], bins=20)

In 55% of the countries the average is less then 10 suisdes per year. 

# <a id='load_libraries'>Number of Suicides per Year</a>

In [ ]:
pop = Suicide.groupby('year').mean()
pop['Year'] = pop.index
dis = sns.barplot(x=pop['Year'], y=pop['suicides/100k pop'], palette="rocket")
plt.title('Mean number of Suicide per Year', fontsize=20)

for item in dis.get_xticklabels():
    item.set_rotation(45)
rcParams['figure.figsize'] = 12,3    
dis.plot()
#pop['suicides/100k pop'].max()

The average number of suicides per year increased up to 1995 (max: 15). From 1996 to 2015  the number went down to 10. There is an incresae in 2016, but as we will see later it is due to reduction in the amount of sampled coutries during these years.

# <a id='load_libraries'>Suicides per generation</a>
Gen Z, iGen, or Centennials: Born 1996 – TBD.

Millennials or Gen Y: Born 1977 – 1995.

Generation X: Born 1965 – 1976.

Baby Boomers: Born 1946 – 1964.

Traditionalists or Silent Generation: Born 1945 and before.

G.I Generations - 1900 -1920

In [ ]:
gen = Suicide.groupby('generation').mean()
rcParams['figure.figsize'] = 12,3
x = ['Generation Z', 'Millenials', 'Generation X', 'Boomers' ,'Silent', 'G.I. Generation']
gen = gen.reindex(x)
gen['Gen'] = gen.index
dis = sns.barplot(x=gen['Gen'], y=gen['suicides/100k pop'], palette="rocket")
plt.title('Mean number of Suicide per generetion', fontsize=20)

for item in dis.get_xticklabels():
    item.set_rotation(45)
rcParams['figure.figsize'] = 10,3    
dis.plot()

In [ ]:
dis = sns.barplot(x=gen['Gen'], y=gen['gdp_per_capita ($)'], palette="rocket")
plt.title('Mean size of GDP per capita per generetion', fontsize=20)

for item in dis.get_xticklabels():
    item.set_rotation(45)
rcParams['figure.figsize'] = 10,3    
dis.plot()

Gen Z, iGen, or Centennials: Born 1996 – TBD.

Millennials or Gen Y: Born 1977 – 1995.

Generation X: Born 1965 – 1976.

Baby Boomers: Born 1946 – 1964.

Traditionalists or Silent Generation: Born 1945 and before.

G.I Generations - 1900 -1920



# <a id='load_libraries'>Age distribution across all countries</a>


In [ ]:
pop = Suicide.groupby('age').sum()
pop['Age'] = pop.index
tmp = ['5-14 years', '15-24 years', '25-34 years', '35-54 years' ,'55-74 years', '75+ years']
pop = pop.reindex(tmp)
dis = sns.barplot(x=pop['Age'], y=pop['suicides/100k pop'], color="blue")
plt.title('Age distribution across all countries', fontsize=20)
rcParams['figure.figsize'] = 10,5
for item in dis.get_xticklabels():
    item.set_rotation(45)
    
dis.plot()

The number of suicides increases as age increases. Majority of the suicides are from the age of 75+ years. The precnetage is actually higher, since in most countries the average life span is less then 75 years.



In [ ]:
print(pop['suicides/100k pop'])

# <a id='load_libraries'>Male vs Female - number of suicides</a>

In [ ]:
g = Suicide.groupby('sex').sum()['suicides_no']
g = g.reset_index()
rcParams['figure.figsize'] = 5,5
dis = sns.barplot(x=g['sex'], y=g['suicides_no'], color="green")

The number of man suicides is ~5 time highr them female. It is a very big diffrence. There is not enough data to explain it.

#<a id='load_libraries'>GDP per  Capita per Country</a>

In [ ]:
gdp = Suicide.groupby('country').mean()
rcParams['figure.figsize'] = 15,3
f, axes = plt.subplots(1, 3)
sns.distplot(gdp['gdp_per_capita ($)'], bins=30,ax=axes[0]);
x = gdp[gdp['gdp_per_capita ($)'] < 9000 ]
sns.distplot(x['gdp_per_capita ($)'], bins=30,ax=axes[1]);
x1 = gdp[gdp['gdp_per_capita ($)'] > 9000 ]
sns.distplot(x1['gdp_per_capita ($)'], bins=30,ax=axes[2]);

It can be observed that 50% of the countries has a GDP that is less then 9000.


Heat map to check correlation

In [ ]:
Suicide['gdp'] = Suicide[' gdp_for_year ($) '].str.replace(',','')
Suicide['gdp'] = Suicide['gdp'].astype(int)
#f, axes = plt.subplots(2, 1)
rcParams['figure.figsize'] = 10,5
#old = Suicide[Suicide.age == "75+ years"]
sns.heatmap(Suicide.corr(),annot=True,linewidth = 0.5, cmap='coolwarm')
#sns.heatmap(Suicide.corr(method='spearman'),annot=True,linewidth = 0.5, cmap='coolwarm', ax=axes[1])

From the correlation (leaner only) we can observe the follwing possbile correlations:
https://www.dummies.com/education/math/statistics/how-to-interpret-a-correlation-coefficient-r/ - explanation.
1. **Suicde_number - gdp    - 0.43** ( A moderate uphill (positive) relationship) - make sense that the bigger the country, the bigger the total number of suicides.
2. **Population - gdp - 0.71** - (  A strong uphill (positive) linear relationship ) - make sense that the bigger the country, the bigger is the market value.
3. **Population - Suicde no - 0.62**.  it make sense. The bigger the poulation the bigger the absolute number of suicides.
4. **HDI - gdp_per_capita - 0.77** - (A strong uphill (positive) - linear relationship) we will show the correlation later.



# <a id='load_libraries'>1. Ave Suicides per 100K pop vs GDP </a>

In [ ]:
df = Suicide.groupby('country').mean()
#df = Suicide.groupby('year').mean()
rcParams['figure.figsize'] = 15,3
df = df[df['suicides/100k pop'] < 10]
sns.jointplot(x="gdp_per_capita ($)", y="suicides/100k pop", data=df)

There is no correlation. We can see that majority of samples are in coutries with GDP less then 20000$ and less then avarage of 10 suicides per year.

In [ ]:
rcParams['figure.figsize'] = 15,10
df = df[df["gdp_per_capita ($)"] < 5000 ]
sns.jointplot(x=df["gdp_per_capita ($)"], y=df["suicides/100k pop"], kind="hex", color="#4CB391")

Nerrow the view to countries with GDP that is less then 5000. Majority is less then average of 10 suicides per 100k population.

Nerrow the view to countries with GDP that is less then 8000. Majority is less then average of 15 suicides per 100k population.

In [ ]:
rcParams['figure.figsize'] = 15,10
tmp = df[df["gdp_per_capita ($)"] < 9000 ]
sns.jointplot(x=tmp["gdp_per_capita ($)"], y=tmp["suicides/100k pop"], kind="hex", color="#4CB391")

Nerrow the view to countries with GDP that is higher then 60000.  The average number of suicides per 100k population is capped ~15 people per year. (rich countries)

# <a id='load_libraries'>2.Distribution of suicides male and female in different age range and GDP </a>

In [ ]:
df = Suicide[Suicide.age == "75+ years"]
rcParams['figure.figsize'] = 10,3
sns.scatterplot(x=df["gdp_per_capita ($)"], y=df["suicides/100k pop"], hue='sex', data=df)

Age 75+ years.

In [ ]:
df = Suicide[Suicide.age == "55-74 years"]
rcParams['figure.figsize'] = 10,3
sns.scatterplot(x="gdp_per_capita ($)", y="suicides/100k pop", hue='sex', data=df)

Age 55-74 years.

In [ ]:
df = Suicide[Suicide.age == "35-54 years"]
rcParams['figure.figsize'] = 10,3
sns.scatterplot(x="gdp_per_capita ($)", y="suicides/100k pop", hue='sex', data=df)

Age 35-54 years.

In [ ]:
df = Suicide[Suicide.age == "25-34 years"]
rcParams['figure.figsize'] = 10,3
sns.scatterplot(x="gdp_per_capita ($)", y="suicides/100k pop", hue='sex', data=df)

Age 25-34 years.

In [ ]:
df = Suicide[Suicide.age == "15-24 years"]
rcParams['figure.figsize'] = 10,3
sns.scatterplot(x="gdp_per_capita ($)", y="suicides/100k pop", hue='sex', data=df)

Age 15-24 years.

In [ ]:
df = Suicide[Suicide.age == "5-14 years"]
rcParams['figure.figsize'] = 10,3
sns.scatterplot(x="gdp_per_capita ($)", y="suicides/100k pop", hue='sex', data=df)

Age 5-14 years.

The distribuation of male and female is unifinied across all ages range, beside the young age of 5-14. In this age ragne the ratio between the number of suicides male and female is significantly lower. 
We can also see that the ratio between male and female decreases as GDP increase.

In [ ]:
df =Suicide[(Suicide.age == "5-14 years")]
g = df.groupby('sex').sum()['suicides_no']
g = g.reset_index()
rcParams['figure.figsize'] = 5,5
dis = sns.barplot(x=g['sex'], y=g['suicides_no'], color="green")

The ratio between the number of suicides male amd female in the age of 5-14 is ~50%, which is much differnet then 

# <a id='load_libraries'>Trend analysis of GDP and suicides over the years </a>

In [ ]:
rcParams['figure.figsize'] = 12,8
f, axes = plt.subplots(2,3)
df = Suicide.groupby('year').mean()
df['Year']  = df.index
sns.lineplot(x="Year", y='gdp_per_capita ($)', data=df, ax=axes[0,0])
sns.lineplot(x="Year", y="suicides/100k pop" , data=df, ax=axes[1,0])
sns.lineplot(x="Year", y="gdp" , data=df, ax=axes[0,2])
df = Suicide.groupby('year').count()
df.reset_index()
df['Year'] = df.index
df['Country'] = (df['country']/12).astype(int)
sns.lineplot(x="Year", y="Country", data=df, ax=axes[0,1])
df = Suicide.groupby('year').sum()
df.reset_index()
df['Year'] = df.index
sns.lineplot(x="Year", y="population" , data=df, ax=axes[1,1])

From the above graphs we can observe that between the years 2000-2010 the number of countries that took part in the sample were more or less the same (later I will identify the countries that appeared in all years during  this period) 

We can see that the total GDP grow up over the year, the same as the GDP_per_capita (beside small drop in 2009 - related to the big crisis)

We can also see that the normalized number of suicides dropped constantly from 1996 to date.

We can ignore 2016, since the number of countries in the sample in this year dropped significantly.

We can observe that there is a trend. While the mean GDP_per_capita is constantly growing from 2000 to 2010, the mean of the normalized suicides value is decreasing.  

# <a id='load_libraries'>Trend analysis of GDP and suicides over 2000-2010 </a>

In [ ]:
rcParams['figure.figsize'] = 12,8
f, axes = plt.subplots(2,3)
df = Suicide.groupby('year').mean()
df['Year']  = df.index
df = df[(df['Year'] < 2010) & (df['Year'] > 2000)]
sns.lineplot(x="Year", y='gdp_per_capita ($)', data=df, ax=axes[0,0])
sns.lineplot(x="Year", y="suicides/100k pop" , data=df, ax=axes[1,0])
sns.lineplot(x="Year", y="gdp" , data=df, ax=axes[0,2])
df = Suicide.groupby('year').count()
df.reset_index()
df['Year'] = df.index
df = df[(df['Year'] < 2010) & (df['Year'] > 2000)]
df['Country'] = (df['country']/12).astype(int)
sns.lineplot(x="Year", y="Country", data=df, ax=axes[0,1])
df = Suicide.groupby('year').sum()
df.reset_index()
df['Year'] = df.index
df = df[(df['Year'] < 2010) & (df['Year'] > 2000)]
sns.lineplot(x="Year", y="population" , data=df, ax=axes[1,1])

We can observe that there is a trend. While the mean GDP_per_capita is constantly growing from 2000 to 2010, the mean of the normalized suicides value is decreasing.

Now we should review the countries that appeared in all years during this period.


In [ ]:
df2 = Suicide.groupby(['year','country','age']).mean()
df2 = df2.reset_index('country')
df2 = df2.reset_index('age')
tmp = df2[df2.index == 2001]
cset = set(tmp['country'])

for i in range(2002,2011):
  tmp = df2[df2.index == i]
  cset = cset & set(tmp['country'])
  
print("Number of coutries that apears in all years: ", len(cset))

df2 = df2[(df2.index > 1999) & (df2.index < 2011)]
df2 = df2[df2.country.isin(cset) ]

The total number of countries that were sampled in all years during the period of 2000-2010 is 71.

# <a id='load_libraries'>Trend analysis of GDP and suicides over 2000-2010  - 71 countries</a>

In [ ]:
# ------------ Data Analysis --------
rcParams['figure.figsize'] = 10,10
f, axes = plt.subplots(2,1)
df7 = df2.reset_index('year')
df7 = df7.groupby(['year','country','age']).mean()
df7 = df7.reset_index('country')
df7 = df7.reset_index('age')
#df2 = df2[df2['age'] == "75+ years"]
df7['Year']  = df7.index
sns.lineplot(x="Year", y='gdp_per_capita ($)', data=df7, ax=axes[0])
sns.lineplot(x="Year", y="suicides/100k pop" ,data=df7, ax=axes[1])

We can observe that there is a trend. While the mean GDP_per_capita is constantly growing from 2000 to 2010, the mean of the normalized suicides value is decreasing.

We can conclude that there is a relation between the gdp_per_capita and the Suicides/100k pop features. As the economy is growing the mean, number of suicides across all sampled countries is decreasing.

Next we will validate it, by looking at some countires.


In [ ]:
f, axes = plt.subplots(2,1)
df = Suicide[(Suicide['country'] == 'Israel')]
sns.lineplot(x="year", y="suicides/100k pop" , hue='sex', data=df, ax=axes[0]).set_title("Trend analysis of Suicides in Israel")
sns.lineplot(x="year", y="gdp_per_capita ($)" , data=df, ax=axes[1]).set_title("Trend analysis of GDP per Capita in Israel")

The relation between the parameters is as expected in Israel





In [ ]:
f, axes = plt.subplots(2,1)
df = Suicide[(Suicide['country'] == 'United States')]
sns.lineplot(x="year", y="suicides/100k pop" , data=df, hue='sex',ax=axes[0]).set_title("Trend analysis of Suicides in United States")
sns.lineplot(x="year", y="gdp_per_capita ($)" , data=df, ax=axes[1]).set_title("Trend analysis of GDP per Capita in United States")

Ooops. In US the GDP per capita is growing, on the other hand the number Suicides per year is slightly growing.

In [ ]:
f, axes = plt.subplots(2,1)
df = Suicide[(Suicide['country'] == 'Finland') & (Suicide['year'] > 2000) & (Suicide['year'] < 2011)]
sns.lineplot(x="year", y="suicides/100k pop" , data=df, hue='sex',ax=axes[0]).set_title("Trend analysis of Suicides in Finland")
sns.lineplot(x="year", y="gdp_per_capita ($)" , data=df, ax=axes[1]).set_title("Trend analysis of GDP per Capita in Finland")

The relation between the parameters is as expected in Finland.

Since in US the relation is not working as expected we can conlucde that there are probably other parameters that impcat the mean number of suicides per year. 

In the next chapter we will review the Gini Index and try to see if it has an impact on the number Suicides.

# <a id='load_libraries'>The Gini Index</a>
"Standard economic measure of income inequality, based on Lorenz Curve. A society that scores 0.0 on the Gini scale has perfect equality in income distribution. Higher the number over 0 higher the inequality, and the score of 1.0 (or 100) indicates total inequality where only one person corners all the income. It is used also as a measure of other distributional inequalities such as market share. Named after its inventor, the Italian statistician Corrado Gini (1884-1965). Also called Gini coefficient or index of concentration." https://datahub.io/world-bank/si.pov.gini#data"

In [ ]:
Gini = pd.read_csv('../input/gini-index/data_csv.csv', encoding = "ISO-8859-1")
Gini = Gini.rename(index=str, columns={"Country Name": "country", "Year": "year", "Value" : "gini_reported"})
Gini.head()

The Gini Index is between 0 to 100. The higher the Index the higher the inequality in this country.

In [ ]:
df1 = Gini.groupby(['year','country']).mean()
df1 = df1.reset_index('country')
df1['Year'] = df1.index
df1 = df1[df1.country.isin(cset) ]
df1['country'].nunique()
df1.head()

Group the the data by Year and get the mean of the Index values per year.

In [ ]:
rcParams['figure.figsize'] = 20,5
df4 = df1.groupby('year').mean()
df4['Year'] = df4.index
dis = sns.barplot(x=df4['Year'], y=df4['gini_reported'], palette="rocket")
plt.title('Mean Gini Index per Year', fontsize=20)

for item in dis.get_xticklabels():
    item.set_rotation(45)
rcParams['figure.figsize'] = 10,3    
dis.plot()

We can see that the Index is gradually going down over the years

In [ ]:
df1.info()

In [ ]:
df1.describe()

In [ ]:
df1 = df1[(df1['Year'] > 1999) & (df1['Year'] < 2011)]
df2['gini_reproted'] = 0.0 
df2 = df2.reset_index()
for index, row in df2.iterrows():
  row_gini = df1.loc[( df1['Year'] == row['year']) & (df1['country'] == row['country'])]
  if row_gini['Year'].empty == False: 
    df2.at[index,'gini_reproted'] = row_gini['gini_reported']

Add the column 'gini_reported' to the Suicides’ Rate database. a

In [ ]:
df2  = df2[df2['gini_reproted'] != 0]

Remove al lteh columns that doesn't include the Gini Index

In [ ]:
print(df2['country'].nunique())
df2.head()

There are 51 countries i nthe combined tables during the years of 2000-2010.

In [ ]:
#df3 = df2[df2['gini_reproted'] < 40]
sns.jointplot(x="gini_reproted", y="suicides/100k pop", data=df2)

There is no relation between the two parameters. most of the samples are the area where the gini index is lower than 40 and the Suicides rate is below 30.

Next we will look at the Trend.

In [ ]:
rcParams['figure.figsize'] = 10,12
f, axes = plt.subplots(3,1)
df = df2[df2['country'] == 'Argentina']
sns.lineplot(x="year", y="suicides/100k pop" , data=df,ax=axes[0]).set_title("Trend analysis of Suicides in Argentina")
sns.lineplot(x="year", y="gdp_per_capita ($)" , data=df, ax=axes[1]).set_title("Trend analysis of GDP per Capita in Argentina")
sns.lineplot(x="year", y='gini_reproted' , data=df, ax=axes[2]).set_title("Trend analysis of Gini in Argentina")

In Argentina we can see that the gini Index is going dwon, the gdp per capita is goin up and the Suicdes rate is going down. 

In [ ]:
rcParams['figure.figsize'] = 10,12
f, axes = plt.subplots(3,1)
df = df2[df2['country'] == 'United States']
sns.lineplot(x="year", y="suicides/100k pop" , data=df,ax=axes[0]).set_title("Trend analysis of Suicides in'United States")
sns.lineplot(x="year", y="gdp_per_capita ($)" , data=df, ax=axes[1]).set_title("Trend analysis of GDP per Capita in 'United States")
sns.lineplot(x="year", y='gini_reproted' , data=df, ax=axes[2]).set_title("Trend analysis of Gini in 'United States")

In USA we can see that the gini Index is high (above the mean), while the gdp per capita is goin up and the Suicdes rate is going down. 




# <a id='load_libraries'>Trend analysis of GDP, suicides and Gini Index over 2000-2010  - 51 countries</a>

In [ ]:
rcParams['figure.figsize'] = 10,10
f, axes = plt.subplots(3,1)
df3 = df2.groupby(['year','country']).mean()
df3 = df3.reset_index('country')
df3['Year']  = df3.index
#df3 = df3[df3['gini_reproted'] < 30]
sns.lineplot(x="Year", y='gini_reproted', data=df3, ax=axes[0])
sns.lineplot(x="Year", y="gdp_per_capita ($)"  , data=df3, ax=axes[1])
sns.lineplot(x="Year", y="suicides/100k pop" , data=df3, ax=axes[2])

# <a id='load_libraries'>Summery</a>
We can conclude the following fro mthe anlysis:


1. The total number of suicides Male is five times bigger then female.
2. In early ages (5-14) the ratio between the number of Male suicides and female is two.
3. There is a trend relation between gdp_per_capita and normalized number of suicides per year.
4. The trend relation between these two parameters is not working for all countries. I took as an example another parameter called Gini Index and showed that there is also a trend relation between this parameter and the Suicides rate.
5. I also looked at some countries as a sample to validate the relation. For example in US even though the GDP per capita is going up, the Suicides rate is going up. The Gini Index during this period, is not improving and is higher than the mean.

It is clear that there are other factors besides GDP and Gini Index, which affect the Suicides rate. Factors such as happiness Index, cultures, Suicides reports, weather, war zones, communities, etc could also affect the Suicides rate.  


